**Этап 1:**

1. Прогноз поставок

2. Цель: обеспечить точную подачу товаров на склады маркетплейса с учётом спроса, сезонности, цен, логистики и ограничений.

3. Функциональность:
  - Построение прогноза спроса по каждому артикулу по дням (горизонт: 7–30 дней)
  - Учет сезонных факторов, акций, календарных праздников, промо, выходных
  - Слежение за остатками, вычисление даты исчерпания при текущем спросе.
  - Учет складских ограничений по API Ozon (доступность мест, FBO-зоны).
  - Расчёт рекомендованного объёма поставки.
  - Экспорт таблиц в Excel/CSV для логистики.

4. Источники данных:
 - История продаж за 12+ месяцев (по дням).
 - Остатки по каждому SKU.
 - API Ozon: остатки, движение товаров, доступность поставок.
 - Календарь акций и праздников (вводится вручную).
 - Информация по складам и лимитам (из API или вручную).

В архитектуре TFT можно выделить следующие блоки:

1. **Static Covariate Encoders** - обработка статических (неизменяемых) признаков.
Помощь модели в учете постоянных особенностей объектов
2. **Variable Selection** - отбор переменных признаков. Избавляется от шумных признаков, которые могут мешать модели
3. **LSTM** - для краткосрочных паттернов. Обрабатывает локальные временные зависимости, улавливает недавние тренды и сезонность
4. **Interpretable Multi-head Attention** - для долгосрочных паттернов. Объединяет информацию из разных временных масштабов, находит сложные долгосрочные зависимости в данных
5. **Quantile Outputs** - прогнозирование квантилей. Оценивает неопределенность предсказаний через различные квантили, позволяет понять возможный разброс прогнозируемых значений

**Статичные признаки проходят следующий путь:**

1) **Категориальные данные преобразуются в эмбеддинги**, а числовые данные используются как есть

2) Все **статичные признаки проходят через VSN слой**

3) Затем создаются **контекстные вектора для передачи в последующие слои модели**





1.   с(s) - Основной контекст статичных признаков - отбор признаков
2.   c(c) - Начальное состояние cell state для LSTM
3.   c(h) - Начальное hidden state для LSTM - поиск краткосрочных паттернов
4.   c(e) - Обогащает выходы LSTM перед слоем трансформеров - поиск долгосрочных паттернов




**Локальный прогноз (LSTM)**

В архитектуре Temporal Fusion Transformer обработка временных зависимостей происходит на двух уровнях: локальном и глобальном. Такой двойной подход позволяет модели эффективно улавливать как краткосрочные паттерны, так и долгосрочные взаимосвязи в данных.

На локальном уровне используется уже знакомая нам  LSTM (Long Short-Term Memory). Представьте, что вы анализируете продажи в магазине: **LSTM отслеживает последовательные изменения в данных, например, как продажи меняются от дня к дню или от недели к неделе. Эта часть модели справляется с выявлением краткосрочных зависимостей и локальных паттернов.**

Механизм фильтрации (gate) и skip connections после LSTM позволяют игнорировать **неиспользуемые компоненты архитектуры, адаптируя глубину и сложность сети для работы с различными наборами данных и сценариями.**

В отличие от LSTM, **Multi-head attention способен напрямую связывать события, происходящие в разные моменты времени, даже если они разделены большими промежутками.** Это похоже на то, как опытный аналитик может заметить, что всплеск продаж в декабре этого года похож на прошлогодний.

Механизм внимания особенно **полезен для выявления сезонности, долгосрочных трендов и повторяющихся паттернов. Например, он может обнаружить, что пик продаж определенных товаров наблюдается каждый год перед началом учебного года, или что существует устойчивая корреляция между продажами разных категорий товаров.**

Рассмотрим пример, представьте, что вы анализируете продажи магазина:

- Первая голова замечает, что продажи выше по выходным

- Вторая отслеживает рост в начале каждого месяца (после зарплаты)

- Третья фокусируется на сезонных пиках (например, перед Новым годом)

**Суть attention в том, чтобы научить модель фокусироваться на важных частях входных данных при генерации выхода. По сути, *attention позволяет модели динамически создавать связи между разными частями входных данных*, *вместо того чтобы работать с фиксированным контекстом*, как это делает LSTM.**

In [2]:
import math
from typing import Optional, Tuple

import torch
import torch.nn as nn


class CustomLSTM(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Forget gate parameters
        self.weight_forget_x = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.weight_forget_h = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bias_forget = nn.Parameter(torch.Tensor(hidden_size))

        # Input gate parameters
        self.weight_input_x = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.weight_input_h = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bias_input = nn.Parameter(torch.Tensor(hidden_size))

        # Cell gate parameters
        self.weight_cell_x = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.weight_cell_h = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bias_cell = nn.Parameter(torch.Tensor(hidden_size))

        # Output gate parameters
        self.weight_output_x = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.weight_output_h = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bias_output = nn.Parameter(torch.Tensor(hidden_size))

        self.init_weights()

    def init_weights(self) -> None:
        """Initialize weights using uniform distribution."""
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(
        self,
        x: torch.Tensor,
        initial_states: Optional[Tuple[torch.Tensor, torch.Tensor]] = None
    ) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """
        Forward pass of LSTM.

        Args:
            x: Input tensor of shape (batch_size, sequence_length, input_size)
            initial_states: Tuple of initial hidden and cell states
                          Each of shape (batch_size, hidden_size)

        Returns:
            tuple: (output_sequence, (final_hidden_state, final_cell_state))
                  output_sequence: tensor of shape (batch_size, sequence_length, hidden_size)
                  final_hidden_state: tensor of shape (batch_size, hidden_size)
                  final_cell_state: tensor of shape (batch_size, hidden_size)
        """
        batch_size, sequence_length, _ = x.size()
        hidden_sequence = []

        if initial_states is None:
            hidden_state = torch.zeros(batch_size, self.hidden_size).to(x.device)
            cell_state = torch.zeros(batch_size, self.hidden_size).to(x.device)
        else:
            hidden_state, cell_state = initial_states

        for t in range(sequence_length):
            curr_x = x[:, t, :]

            # Gates computation

            #ШАГ 1. Первый шаг в LSTM – определить, какую информацию можно выбросить из состояния ячейки.
            #Если вес равен 1, то предыдущие состояние полностью сохранится, а если forget layer gate вернул 0, то предыдущее состояние будет "забыто".

            """
            Для задачи предсказания временных рядов, можно представить, что модель перестает учитывать новогодние праздники, потому что требуется прогноз на середину января.
            """
            forget_gate = torch.sigmoid(curr_x @ self.weight_forget_x + hidden_state @ self.weight_forget_h + self.bias_forget)

            #ШАГ 2. решить, какая новая информация будет храниться в состоянии текущей ячейки. Этот этап состоит из двух частей.
            #Сначала сигмоидальный слой входного фильтра (input layer gate) определяет, какие значения следует обновить.
            #Затем tanh-слой строит вектор C(t)^~ кандидатов нового состояния, которые потом могут быть к состоянию ячейки C(t).

            #Получение вектора состояния
            """
            Можно представить, как модель замечает замедление тренда у временного ряда.
            """
            input_gate = torch.sigmoid(curr_x @ self.weight_input_x + hidden_state @ self.weight_input_h + self.bias_input)

            #ШАГ 3. Теперь очередь обновить старое состояние ячейки. Из C(t-1) в C(t)

            """
            Тут модель может забыть про новогодние праздники и делать прогнозы на обычные дни и с учетом замедленного тренда.
            """

            cell_candidate = torch.tanh(curr_x @ self.weight_cell_x + hidden_state @ self.weight_cell_h + self.bias_cell)

            #ШАГ 4.
            #Остается решить, что мы собираемся выводить в качестве скрытого состояния для следующего блока LSTM.

            """
            Для задачи прогноза временного ряда модель может передать дальше информацию о снижении тренда.
            """

            output_gate = torch.sigmoid(curr_x @ self.weight_output_x + hidden_state @ self.weight_output_h + self.bias_output)

            # States update
            cell_state = forget_gate * cell_state + input_gate * cell_candidate
            hidden_state = output_gate * torch.tanh(cell_state)

            hidden_sequence.append(hidden_state.unsqueeze(0))

        # Stack and reshape hidden sequence
        hidden_sequence = torch.cat(hidden_sequence, dim=0)
        hidden_sequence = hidden_sequence.transpose(0, 1).contiguous()

        return hidden_sequence, (hidden_state, cell_state)



*   residual = x (если размерности совпадают)
*   h = ELU(**Linear(x)**) → [0.8, -0.2, 1.5]


*   h = Linear(h) → [0.5, 0.3, -0.1]
*   h = Dropout(h) → [0.5, 0.3, -0.1]



*   h = GLU(h) → [0.5*σ(0.5), 0.3*σ(0.3), -0.1*σ(-0.1)] ≈ [0.31, 0.19, -0.05].
*   h = h + residual → [1.31, -0.31, 1.95]


x = LayerNorm(h) → Нормализованный выход.

Gated Residual Network (GRN)  - это блок, который позволяет сети "решать" насколько сложные преобразования нужно применить к входным данным, вплоть до полного пропуска преобразований, если они не нужны. Это особенно полезно, когда мы заранее не знаем, какие входные переменные важны.


In [3]:
import torch
from torch import nn


class GatedLinearUnit(nn.Module):
    """
    Gated Linear Unit implementation.

    Args:
        input_size (int): Size of input features
        output_size (int): Size of output features

    Returns:
        torch.Tensor: Output tensor after applying GLU
    """
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.sigmoid = nn.Linear(input_size, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the GLU.

        Args:
            x (torch.Tensor): Input tensor

        Returns:
            torch.Tensor: Result of linear transformation multiplied by sigmoid gate
        """
        return self.linear(x) * torch.sigmoid(self.sigmoid(x))


class GatedResidualNetwork(nn.Module):
    """
    Gated Residual Network implementation.

    Args:
        input_size (int): Size of input features
        hidden_size (int): Size of hidden layer
        dropout_rate (float): Dropout rate for regularization

    Returns:
        torch.Tensor: Output tensor after processing through GRN
    """
    def __init__(self, input_size, hidden_size, dropout_rate):
        super().__init__()

        self.hidden_size = hidden_size

        self.elu_dense = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ELU()
        )

        self.linear_dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.glu = GatedLinearUnit(hidden_size, hidden_size)
        self.layer_norm = nn.LayerNorm(hidden_size)

        self.project = nn.Linear(input_size, hidden_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the GRN.

        Args:
            x (torch.Tensor): Input tensor

        Returns:
            torch.Tensor: Processed tensor

         Слой с функций активации ELU
         Механизм фильтрации GLU
         Residual (skip) connections
         Нормализация LayerNorm
         Дополнительно используется Dropout после линейного слоя.

         Подготовка residual-пути (project или identity).
         Основной блок:
         ELU → Linear → Dropout → GLU.
         Добавление residual-связи и нормализация.
        """
        if x.shape[-1] != self.hidden_size:
            residual = self.project(x)
        else:
            residual = x

        # слой активации ELU
        # ELU transformation
        h = self.elu_dense(x)

        # Linear transformation
        h = self.linear_dense(h)

        # Apply dropout
        h = self.dropout(h)

        # Apply GLU gating mechanism
        h = self.glu(h)

        # Add residual connection
        h = h + residual

        # Apply layer normalization
        x = self.layer_norm(h)
        return x

Variable Selection Network (VSN) - определяет важность (выбирает) каждого признака, **помогая модели сфокусироваться на действительно важных фичах.** VSN блоки действуют как умные фильтры, оценивающие важность каждого признака и удаляющие **ненужные-шумные данные которые могут мешать модели.** В TFT это самый первый слой через который проходят входные данные.

Используя **VSN модель автоматически определяет важность каждого признака.**

Представьте, что вы анализируете продажи магазина: одни факторы, например, сезонность или день недели, могут оказывать сильное влияние на продажи, тогда как другие, скажем, влажность воздуха, могут быть менее значимыми.

In [ ]:
import torch
from torch import nn

from typing import List


class GatedLinearUnit(nn.Module):
    """
    Gated Linear Unit implementation.

    Args:
        input_size (int): Size of input features
        output_size (int): Size of output features

    Returns:
        torch.Tensor: Output tensor after applying GLU
    """
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.sigmoid = nn.Linear(input_size, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the GLU.

        Args:
            x (torch.Tensor): Input tensor

        Returns:
            torch.Tensor: Result of linear transformation multiplied by sigmoid gate
        """
        return self.linear(x) * torch.sigmoid(self.sigmoid(x))


class GatedResidualNetwork(nn.Module):
    """
    Gated Residual Network implementation.

    Args:
        input_size (int): Size of input features
        hidden_size (int): Size of hidden layer
        dropout_rate (float): Dropout rate for regularization

    Returns:
        torch.Tensor: Output tensor after processing through GRN
    """
    def __init__(self, input_size, hidden_size, dropout_rate):
        super().__init__()

        self.hidden_size = hidden_size

        self.elu_dense = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ELU()
        )

        self.linear_dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.glu = GatedLinearUnit(hidden_size, hidden_size)
        self.layer_norm = nn.LayerNorm(hidden_size)

        self.project = nn.Linear(input_size, hidden_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the GRN.

        Args:
            x (torch.Tensor): Input tensor

        Returns:
            torch.Tensor: Processed tensor
        """
        if x.shape[-1] != self.hidden_size:
            residual = self.project(x)
        else:
            residual = x

        x = self.elu_dense(x)
        x = self.linear_dense(x)
        x = self.dropout(x)
        x = self.glu(x)
        x = x + residual
        x = self.layer_norm(x)

        return x


class VariableSelection(nn.Module):
    """
    Variable Selection Network using Gated Residual Networks.

    Args:
        input_size (int): Size of input features
        hidden_size (int): Size of hidden layer
        dropout_rate (float): Dropout rate for regularization

    Returns:
        torch.Tensor: Weighted combination of processed features
    """
    def __init__(self, input_size: int, hidden_size: int, dropout_rate: float) -> None:
        super().__init__()
        self.num_features = input_size  # number of input features
        self.grns = nn.ModuleList()

        # Create a GRN for each feature independently
        for _ in range(self.num_features):
            grn = GatedResidualNetwork(hidden_size, hidden_size, dropout_rate)
            self.grns.append(grn)

        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(hidden_size * self.num_features,
                                             hidden_size,
                                             dropout_rate)
        self.softmax = nn.Linear(hidden_size, self.num_features)

    def forward(self, inputs: List[torch.Tensor]) -> torch.Tensor:
        """
        Forward pass of the Variable Selection Network.

        Args:
            inputs (List[torch.Tensor]): List of input feature tensors

        Returns:
            torch.Tensor: Weighted combination of processed features
        """
        # Concatenated inputs for features weights
        v = torch.cat(inputs, dim=-1)

        # YOUR CODE HERE
        v = self.grn_concat(v)
        v = self.softmax(v)
        v = torch.softmax(v, dim=-1)
        v = v.unsqueeze(-1)

        # Add GRN for each feature
        x = []
        for idx, input in enumerate(inputs):
            # YOUR CODE HERE
            out = self.grns[idx](input)
            x.append(out)

        x = torch.stack(x, dim=1)

        # Compute weighted sum
        result = torch.matmul(v.transpose(-2, -1), x).squeeze(1)
        return result

Обрати внимание, что **каждый шаг временного ряда**, как из исторических данных (past inputs), **так и из данных для будущего (know future inputs), передается в свой отдельный слой VSN.** На последующих схемах мы будем опускать этот момент для упрощения визуализации и восприятия.